In [1]:
import pandas as pd
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from torchnlp.encoders.text import StaticTokenizerEncoder, stack_and_pad_tensors, pad_tensor
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from time import time
from collections import defaultdict

C:\Users\wojtek\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\wojtek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\wojtek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [3]:
class TBSADataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len=128):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = self.targets[idx]

        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          padding='max_length',
          return_token_type_ids=False,
          max_length=self.max_len,
          return_attention_mask=True,
          return_tensors='pt',
          truncation='only_first'
        )

        return {
          'review_text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }


In [4]:
def create_data_loader(df):
    ds = TBSADataset(
        texts=df.text.to_numpy(),
        targets=df.sentiment.to_numpy(),
        tokenizer=tokenizer,
        max_len=128
    )
    return DataLoader(
        ds,
        batch_size=32,
        shuffle=True
    )

In [13]:
df = pd.read_csv('data/data_no_tweet.tsv', sep='\t')
df.sentiment += 1
df_train, df_test = train_test_split(
  df,
  test_size=0.2,
  random_state=42
)

df_train, df_eval = train_test_split(
  df_train,
  test_size=0.2,
  random_state=42
)

In [6]:
train_dl = create_data_loader(df_train)

In [7]:
test_dl = create_data_loader(df_test)

In [8]:
eval_dl = create_data_loader(df_eval)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

In [5]:
class TBSA(nn.Module):

    def __init__(self):
        super(TBSA, self).__init__()
        self.transformer = AutoModel.from_pretrained('distilbert-base-uncased', return_dict=False)
        
        self.drop = nn.Dropout(p=0.2)
        self.out = nn.Linear(self.transformer.config.hidden_size, 3)

    def forward(self, input_ids, attention_mask):
        pooled_output = self.transformer(
          input_ids=input_ids,
          attention_mask=attention_mask,
        )[0][:, 0, :]
        output = self.drop(pooled_output)
        return self.out(output)

In [6]:
loss_fn = nn.CrossEntropyLoss().to(device)
model = TBSA().to(device)
optimizer = AdamW(model.parameters(), lr=5e-5, correct_bias=False)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=int(len(train_dl) * 1)
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NameError: name 'train_dl' is not defined

In [19]:
def train_epoch(model, train_dl, acc_steps=1):
    model = model.train()
    losses = []
    avg_losses = []
    temp_preds = []
    temp_targets = []
    avg_accs = []
    acc_losses = []
    correct_predictions = 0
    i = 0
    t0 = time()
    train_size = len(df_train)
    for d in train_dl:
        input_ids = d["input_ids"].to(device)
        targets = d["targets"].to(device).view(-1)

        attention_mask = d["attention_mask"].to(device)
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        preds = outputs.argmax(1, keepdim = True).view(-1)
        loss = loss_fn(outputs, targets)
        loss = loss / acc_steps
        correct_predictions += torch.sum(preds == targets)

        temp_preds += preds.cpu().tolist()
        temp_targets += targets.cpu().tolist()

        acc_losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        i += 1
        if i % acc_steps == 0:
            losses.append(np.mean(acc_losses))
            acc_losses = []
            optimizer.step()
            #scheduler.step()
            optimizer.zero_grad()
        if i % (100 * acc_steps) == 0:
            acc = 0
            try:
                acc = accuracy_score(temp_targets, temp_preds)
            except ValueError:
                pass
            temp_preds = []
            temp_targets = []

            avg_accs.append(acc)
            avg_losses.append(np.mean(losses[i-100:i]))
            print(i, 'iters, auroc, loss, time : ', avg_accs[-1], avg_losses[-1], time()-t0)

    return correct_predictions.double() / train_size, np.mean(losses), avg_losses, avg_accs

In [20]:
def eval_model(model, eval_dl):
    model = model.eval()
    losses = []
    temp_preds = []
    temp_targets = []   
    correct_predictions = 0
    eval_size = len(df_eval)
    with torch.no_grad():
        for d in eval_dl:
            input_ids = d["input_ids"].to(device)
            targets = d["targets"].to(device).view(-1)
            outputs = torch.zeros_like(targets)

            attention_mask = d["attention_mask"].to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            preds = outputs.argmax(1, keepdim = True).view(-1)
            loss = loss_fn(outputs, targets)
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

            temp_preds += preds.cpu().tolist()
            temp_targets += targets.cpu().tolist()


    acc = 0
    try:
        acc = accuracy_score(temp_targets, temp_preds)
    except ValueError:
        pass

    return correct_predictions.double() / eval_size, np.mean(losses)


In [21]:
def get_predictions(model, test_dl):
    model = model.eval()
    review_texts = []
    predictions = []
    prediction_probs = []
    real_values = []
    with torch.no_grad():
        for d in test_dl:
            texts = d["review_text"]
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            preds = outputs.argmax(1, keepdim = True).view(-1)
            review_texts.extend(texts)
            predictions.extend(preds)
            real_values.extend(targets)

    predictions = torch.stack(predictions).cpu()
    real_values = torch.stack(real_values).cpu()
    return review_texts, predictions, real_values


In [16]:
history = defaultdict(list)
best_acc = 0
epochs = 5

In [22]:
def run_epoch(epoch, best_acc):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)
    train_acc, train_loss, train_avg_losses, avg_accs = train_epoch(model, train_dl)
    
    print(f'Train loss {train_loss} accuracy {train_acc}')
    
    val_acc, val_loss = eval_model(model, eval_dl)

    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()
    
    history['train_acc'].append(train_acc)
    history['train_loss'] += train_avg_losses
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_acc:
        best_acc = val_acc
    return best_acc


In [25]:
for epoch in range(epochs):
    run_epoch(epoch, best_acc)

Epoch 1/5
----------
100 iters, auroc, loss, time :  0.7096875 0.7012013110518456 23.46341633796692
Train loss 0.6748644018270931 accuracy 0.7217368961973278
Val   loss 0.5442714143183923 accuracy 0.7882836587872559

Epoch 2/5
----------
100 iters, auroc, loss, time :  0.8121875 0.4827631662786007 23.28145956993103
Train loss 0.48346972502157337 accuracy 0.8111510791366907
Val   loss 0.5431040544663707 accuracy 0.7882836587872559

Epoch 3/5
----------
100 iters, auroc, loss, time :  0.80375 0.5017260302603245 23.578564882278442
Train loss 0.495240148462233 accuracy 0.8060123329907503
Val   loss 0.5385278895978005 accuracy 0.7882836587872559

Epoch 4/5
----------
100 iters, auroc, loss, time :  0.8059375 0.4934819088876247 23.377612829208374
Train loss 0.48639885287304396 accuracy 0.8101233299075026
Val   loss 0.5365809346399 accuracy 0.7882836587872559

Epoch 5/5
----------
100 iters, auroc, loss, time :  0.8153125 0.4842222927510738 23.436198234558105
Train loss 0.4895379302687332 acc

In [26]:
y_review_texts, y_pred, y_test = get_predictions(model, test_dl)
print(accuracy_score(y_test, y_pred))

0.80936729663106


In [27]:
f1_score(y_test, y_pred, average='macro')

0.685866444605009

In [28]:
confusion_matrix(y_test, y_pred)

array([[186,  10,  66],
       [ 49,  55,  51],
       [ 44,  12, 744]], dtype=int64)

In [15]:
df['target_text'] = df.apply(lambda x: x.text + ' [SEP] ' + x.target, axis=1)

# TBSA

In [23]:
def create_tb_data_loader(df):
    ds = TBSADataset(
        texts=df.target_text.to_numpy(),
        targets=df.sentiment.to_numpy(),
        tokenizer=tokenizer,
        max_len=128
    )
    return DataLoader(
        ds,
        batch_size=32,
        shuffle=True
    )

In [24]:
df_train, df_test = train_test_split(
  df,
  test_size=0.2,
  random_state=42
)

df_train, df_eval = train_test_split(
  df_train,
  test_size=0.2,
  random_state=42
)

train_dl = create_tb_data_loader(df_train)

test_dl = create_tb_data_loader(df_test)

eval_dl = create_tb_data_loader(df_eval)

In [25]:
#weights = torch.Tensor(compute_class_weight('balanced', [0,1,2], df_train.sentiment.values))
#weights = weights / weights.sum()
loss_fn = nn.CrossEntropyLoss().to(device)
model = TBSA().to(device)
optimizer = AdamW(model.parameters(), lr=3e-3, correct_bias=False)

#scheduler = get_linear_schedule_with_warmup(
#    optimizer,
#    num_warmup_steps=0,
#    num_training_steps=int(len(train_dl) * 1)
#)
history = defaultdict(list)
best_acc = 0
epochs = 5

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
for epoch in range(epochs):
    run_epoch(epoch, best_acc)

Epoch 1/5
----------
100 iters, auroc, loss, time :  0.556875 1.2376426005363463 11.841158390045166
Train loss 1.1836741117180372 accuracy 0.5637204522096608
Val   loss 0.9462817830424155 accuracy 0.6084275436793423

Epoch 2/5
----------
100 iters, auroc, loss, time :  0.6184375 0.9332586687803268 11.805704832077026
Train loss 0.9337639803769159 accuracy 0.6158787255909558
Val   loss 0.9439062860704237 accuracy 0.6084275436793423

Epoch 3/5
----------
100 iters, auroc, loss, time :  0.6240625 0.919695433974266 11.874210596084595
Train loss 0.9204537028171977 accuracy 0.6228160328879754
Val   loss 0.9325552005921641 accuracy 0.6084275436793423

Epoch 4/5
----------
100 iters, auroc, loss, time :  0.61625 0.927557065486908 11.92819857597351
Train loss 0.9186002216378196 accuracy 0.6228160328879754
Val   loss 0.9888502609345221 accuracy 0.6084275436793423

Epoch 5/5
----------
100 iters, auroc, loss, time :  0.6159375 0.9364933133125305 11.974231481552124
Train loss 0.923064367204416 accu

In [17]:
#batch=128
y_review_texts, y_pred, y_test = get_predictions(model, test_dl)
print(accuracy_score(y_test, y_pred))
f1_score(y_test, y_pred, average='macro')

0.8348397699260477


0.7507956592076185

In [22]:
#batch=64
y_review_texts, y_pred, y_test = get_predictions(model, test_dl)
print(accuracy_score(y_test, y_pred))
f1_score(y_test, y_pred, average='macro')

0.8373048479868529


0.752758858452138

In [28]:
#256
y_review_texts, y_pred, y_test = get_predictions(model, test_dl)
print(accuracy_score(y_test, y_pred))
f1_score(y_test, y_pred, average='macro')

0.8233360723089564


0.7414560150462578

In [25]:
#192
y_review_texts, y_pred, y_test = get_predictions(model, test_dl)
print(accuracy_score(y_test, y_pred))
f1_score(y_test, y_pred, average='macro')

0.8134757600657354


0.7300259866812592

In [20]:
#128
y_review_texts, y_pred, y_test = get_predictions(model, test_dl)
print(accuracy_score(y_test, y_pred))
f1_score(y_test, y_pred, average='macro')

C:\Users\wojtek\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2184: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0.8356614626129828


0.7674470928633134

In [17]:
#96
y_review_texts, y_pred, y_test = get_predictions(model, test_dl)
print(accuracy_score(y_test, y_pred))
f1_score(y_test, y_pred, average='macro')

0.8027937551355793


0.7024931239464111

In [22]:
#64
y_review_texts, y_pred, y_test = get_predictions(model, test_dl)
print(accuracy_score(y_test, y_pred))
f1_score(y_test, y_pred, average='macro')

0.8175842235004108


0.7256547521288529

In [26]:
confusion_matrix(y_test, y_pred)

array([[197,  15,  50],
       [ 33,  78,  44],
       [ 55,  28, 717]], dtype=int64)

In [27]:
#128
y_review_texts, y_pred, y_test = get_predictions(model, test_dl)
print(accuracy_score(y_test, y_pred))
f1_score(y_test, y_pred, average='macro')

0.8299096138044372


0.7577898094622112

In [34]:
y_review_texts, y_pred, y_test = get_predictions(model, test_dl)
print(accuracy_score(y_test, y_pred))
f1_score(y_test, y_pred, average='macro')

0.8069022185702547


0.7206864594941901

In [23]:
df_eval.sentiment.value_counts()

2    592
0    250
1    131
Name: sentiment, dtype: int64

In [29]:
compute_class_weight('balanced', [0,1,2], df_train.sentiment.values)

C:\Users\wojtek\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0, 1, 2], y=[2 2 2 ... 2 2 2] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


array([1.3627451 , 2.51421189, 0.53520352])

In [35]:
weights / weights.sum()

tensor([0.3089, 0.5698, 0.1213])